# Stereo tracking

## Without occlusions

Path in .gitignore for sample without occlusions is in tracking/conveyor_sample_without 
and 

can be downloaded 
<a href="https://dtudk-my.sharepoint.com/personal/evanb_dtu_dk/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fevanb%5Fdtu%5Fdk%2FDocuments%2FCourses%2F31392%2FFinal%5FProject%2Fsample%5FStereo%5Fconveyor%5Fwithout%5Focclusions%2Erar&parent=%2Fpersonal%2Fevanb%5Fdtu%5Fdk%2FDocuments%2FCourses%2F31392%2FFinal%5FProject&ga=1">Here</a>

The whole conveyor belt dataset without occlusions downloaded <a href="https://dtudk-my.sharepoint.com/personal/evanb_dtu_dk/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fevanb%5Fdtu%5Fdk%2FDocuments%2FCourses%2F31392%2FFinal%5FProject%2FStereo%5Fconveyor%5Fwithout%5Focclusions%2Erar&parent=%2Fpersonal%2Fevanb%5Fdtu%5Fdk%2FDocuments%2FCourses%2F31392%2FFinal%5FProject&ga=1">Here</a>

Dataset conveyor with occlusions can be downloaded <a href="https://dtudk-my.sharepoint.com/personal/evanb_dtu_dk/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fevanb%5Fdtu%5Fdk%2FDocuments%2FCourses%2F31392%2FFinal%5FProject%2FStereo%5Fconveyor%5Fwith%5Focclusions%2Erar&parent=%2Fpersonal%2Fevanb%5Fdtu%5Fdk%2FDocuments%2FCourses%2F31392%2FFinal%5FProject&ga=1">Here</a>

First I´ll try sparse optical flow from Week 2. I will track where the object is and then use the depth map from previous task to aquire the depth = Z coordinate


In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
import imutils
import os

#show in popup
# %matplotlib qt
#show inlince
%matplotlib inline

## Functions

### Load images, one by one

In [3]:
def crop(img, h, w):
    center = (w // 2, h // 2)
    #make black everything on  and rotate by -15 degrees
    M = cv2.getRotationMatrix2D(center, -15, 1.0)
    rotated = imutils.rotate(img, -15)
 

    #create mask of black x pixels 350 to 600 and y pixels 350 to 1200
    mask = np.zeros(img.shape[:2], np.uint8)
    mask[350:600, 350:1200] = 255
    #apply mask to rotated image
    masked = cv2.bitwise_and(rotated, rotated, mask=mask)
    #rotate back by 15 degrees
    M = cv2.getRotationMatrix2D(center, 15, 1.0)
    rotated = imutils.rotate(masked, 15)
    return rotated


def optical_flow(path, show_windows=True, no_of_frames=0, display_OF=False, save=False):
    images = glob.glob(path)
    assert images != [], "No images found in {}".format(path)
    
    first_frame = cv2.imread(images[0])
    #show image for one second
    #crop the image from the top, from the bottom, from the left
    h, w = first_frame.shape[:2]
    

    
    print("Dimensions:", first_frame.shape, "\n")
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    prev_gray = crop(prev_gray, h, w)
    
    
    mask = np.zeros_like(first_frame)
    mask[..., 1] = 255

    #write if else statement on one line
    
    if no_of_frames == 0:   no_of_frames = len(images)-1
    else:                   no_of_frames = no_of_frames

    for i in range(1, no_of_frames):
        if i == 1:  prev_frame = first_frame #for displaying only
        else:   
            prev_frame = cv2.imread(images[i-1])

        next_frame = cv2.imread(images[i])
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY) #convert to grayscale
        next_gray = crop(next_gray, h, w)
                #cv2.calcOpticalFlowFarneback(prev,    next,     flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags)   
                #                               (prev_gray, next_gray, None, 0.5,       3,     15,         3,       5,       1.2,        0)           
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None,0.5,       3,     15,         1,       3,       1.2,        0) #calculate dence OF
        # tvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
        # flow = tvl1.calc(prev_gray, next_gray, None)
        #use sparse optical flow instead
        
        mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1]) # Retrieving the magnitude and angle of every pixel
        mask = np.zeros_like(first_frame) # Create empty matrix in dimesions as original image
        mask[..., 1] = 255              # Set image saturation to maximum value as we do not need it
        mask[..., 0] = ang*180/np.pi/2  # Set image hue according to the optical flow direction
        mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        fx, fy = flow[:,:,0], flow[:,:,1]
        v = np.sqrt(fx*fx+fy*fy)

        hsv = np.zeros((first_frame.shape), np.uint8)
        hsv[...,0] = ang*(90/np.pi/2)
        hsv[...,1] = 255
        hsv[...,2] = np.minimum(v*4, 255)
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        gray1 = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray1, 3 , 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)

        cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        # loop over the contours
        for c in cnts:
            M = cv2.moments(c)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            centroid = (cx,cy)
            area = cv2.contourArea(c)
            #print area

            areaTH = 5000

            # if the contour is too small, ignore it
            (x, y, w, h) = cv2.boundingRect(c)
            a = x + w
            b = y + h
            #display text in the window centroid
            if area > areaTH:
                cv2.putText(next_frame, "Area: {}".format(area), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                cv2.putText(next_frame, "Centroid: ({}, {})".format(cx, cy), (x, y+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                cv2.putText(next_frame, "Height: {}".format(h), (x, y+40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                cv2.putText(next_frame, "Width: {}".format(w), (x, y+60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                cv2.rectangle(next_frame, (x, y), (a , b ), (0, 255, 0), 2)
                cv2.circle(next_frame,(cx,cy),5,(0,0,255), -1)

                
            # if (area > areaTH) and (area < areaTH2) and (w < 400) and (h < 600) :
        
        
        if display_OF:  flow = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR) #convert to RGB for display
        prev_gray = next_gray #update from previous frame

        if show_windows: 
            if display_OF:  out = cv2.addWeighted(next_frame, 1, flow, 2, 0)
            else:           out = next_frame
            cv2.imshow('frame', out)
            if save:
                cv2.imwrite("Dense_OF_save/img" + str(i) + ".jpg", out)
        if cv2.waitKey(1) & 0xFF == ord('q'): #stop on q
            break
        print("\rFrame {}/{}".format(i, no_of_frames+1), end="")

    cv2.destroyAllWindows() #close all windows

    
    




### Running the code

In [4]:
#PARAMETERS
display = True  #whether to display the windows
# path_OF = "conveyor_sample_without/left/*.png"    #sample video without occlusions
path_OF = "conveyor_full_without/left/*.png"        #Full video without occlusions
# path_OF = "conveyor_full_with/left/*.png"         #full video with occlusions

display_OF = True   #whether to display the optical flow
no_of_frames = 0    #how many images to use. Use 0 for all
resize = True       #resize for quicker run. BE CAREFUL! Cant use resizing for depth, also OF is not accurate
scale = 0.25         #resize by this factor 
save = False

#optical_flow(path=path_OF, show_windows=display, no_of_frames=no_of_frames, display_OF=display_OF, save=save)

# if save:
#     path = "Dense_OF_save/"
#     #create a video from all of the frames
#     images = [img for img in os.listdir(path) if img.endswith(".jpg")]
#     images = sorted(images)
#     frame = cv2.imread(path + images[0])
#     height, width, layers = frame.shape
#     video = cv2.VideoWriter('Dense_OF_save.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (width,height))
#     for image in images:
#         video.write(cv2.imread(path + image))
#     cv2.destroyAllWindows()
#     video.release()
#     print("\nVideo saved")



## Sparse optical flow

In [5]:
def crop(img, h, w):
    center = (w // 2, h // 2)
    #make black everything on  and rotate by -15 degrees
    M = cv2.getRotationMatrix2D(center, -15, 1.0)
    rotated = imutils.rotate(img, -15)
 

    #create mask of black x pixels 350 to 600 and y pixels 350 to 1200
    mask = np.zeros(img.shape[:2], np.uint8)
    mask[350:600, 350:1200] = 255
    #apply mask to rotated image
    masked = cv2.bitwise_and(rotated, rotated, mask=mask)
    #rotate back by 15 degrees
    M = cv2.getRotationMatrix2D(center, 15, 1.0)
    rotated = imutils.rotate(masked, 15)
    return rotated

def sparse_optical_flow(path, show_windows=True, no_of_frames=0):
    images = glob.glob(path)
    assert images != [], "No images found in {}".format(path)
    
    first_frame = cv2.imread(images[0])
    #show image for one second
    #crop the image from the top, from the bottom, from the left
    h, w = first_frame.shape[:2]

    # # Read the video 
    # cap = cv2.VideoCapture(video_path)

    # Parameters for ShiTomasi corner detection
    feature_params = dict(maxCorners=1000, qualityLevel=0.1, minDistance=7, blockSize=7)

    # Parameters for Lucas Kanade optical flow
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )

    # Create random colors
    color = np.random.randint(0, 255, (100, 3))

    # Take first frame and find corners in it
    # ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    # prev_gray = crop(prev_gray, h, w)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(first_frame)

    if no_of_frames == 0:   no_of_frames = len(images)-1
    else:                   no_of_frames = no_of_frames
    for i in range(1, no_of_frames):

        next_frame = cv2.imread(images[i])
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY) #convert to grayscale
        p0 = cv2.goodFeaturesToTrack(next_gray, mask=None, **feature_params)
        # next_gray = crop(next_gray, h, w)

        # Calculate Optical Flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            prev_gray, next_gray, p0, None, **lk_params
        )
        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the tracks
        for x, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = np.ceil(new.ravel())
            a, b = int(a), int(b)
            c, d = np.ceil(old.ravel())
            c, d = int(c), int(d)
            mask = cv2.line(mask, (a, b), (c, d), color[x].tolist(), 2)
            next_frame = cv2.circle(next_frame, (a, b), 5, color[x].tolist(), -1)

        if show_windows: 
            img = cv2.add(next_frame, mask)
            cv2.imshow('frame', img)
        if cv2.waitKey(1) & 0xFF == ord('q'): #stop on q
            break
        print("\rFrame {}/{}".format(i, no_of_frames+1), end="")
        # Update the previous frame and previous points
        prev_gray = next_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    cv2.destroyAllWindows() #close all windows

        


In [6]:

display = True  #whether to display the windows
# path_OF = "conveyor_sample_without/left/*.png"    #sample video without occlusions
path_OF = "conveyor_full_without/left/*.png"        #Full video without occlusions
# path_OF = "conveyor_full_with/left/*.png"         #full video with occlusions

display_OF = True   #whether to display the optical flow
no_of_frames = 0    #how many images to use. Use 0 for all
resize = True       #resize for quicker run. BE CAREFUL! Cant use resizing for depth, also OF is not accurate
scale = 0.25         #resize by this factor 

sparse_optical_flow(path=path_OF, show_windows=display, no_of_frames=no_of_frames)

Frame 33/1567

## Find corners tracker

In [7]:
import math

class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h, _ = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

def detector(path, show_windows=True, no_of_frames=0):
    images = glob.glob(path)
    assert images != [], "No images found in {}".format(path)

    tracker = EuclideanDistTracker()
    object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

    if no_of_frames == 0:   no_of_frames = len(images)-1
    else:                   no_of_frames = no_of_frames
    for i in range(1, no_of_frames):
        frame = cv2.imread(images[i])
        height, width, _ = frame.shape

        # Extract Region of interest
        roi = frame[250:600, 350:1200]

        # 1. Object Detection
        mask = object_detector.apply(roi)
        _, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detections = []
        for cnt in contours:
            # Calculate area and remove small elements
            area = cv2.contourArea(cnt)
            if area > 1000:
                #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
                x, y, w, h = cv2.boundingRect(cnt)
                #if the detected object is not inside previously detected object
                if not any(x + w > d[0] and x < d[0] + d[2] and y + h > d[1] and y < d[1] + d[3] for d in detections):
                    #if the area is same or bigger than the previous one we append the new detection
                    if not any(area > d[4] for d in detections):
                        detections.append((x, y, w, h, area))
      

        # 2. Object Tracking
        boxes_ids = tracker.update(detections)
        for box_id in boxes_ids:
            x, y, w, h, id = box_id
            cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

        if show_windows:
            cv2.imshow("roi", roi)
            cv2.imshow("Frame", frame)
            cv2.imshow("Mask", mask)
        if cv2.waitKey(1) & 0xFF == ord('q'): #stop on q
            break


    cv2.destroyAllWindows() #close all windows



In [8]:
#PARAMETERS
import sys
display = True  #whether to display the windows
# path_OF = "conveyor_sample_without/left/*.png"    #sample video without occlusions
path_OF = "conveyor_full_without/left/*.png"        #Full video without occlusions
# path_OF = "conveyor_full_with/left/*.png"         #full video with occlusions

display_OF = True   #whether to display the optical flow
no_of_frames = 0    #how many images to use. Use 0 for all
detector(path=path_OF, show_windows=display, no_of_frames=no_of_frames)


{0: (827, 81)}
{0: (818, 71)}
{0: (811, 65)}
{0: (804, 60)}
{0: (799, 57)}
{0: (794, 53)}
{0: (790, 49)}
{0: (786, 46)}
{0: (782, 46)}
{0: (778, 58)}
{0: (775, 57)}
{0: (773, 58)}
{0: (771, 58)}
{0: (770, 56)}
{1: (738, 61)}
{1: (731, 61)}
{1: (730, 74), 2: (816, 83)}
{2: (816, 83), 1: (730, 74)}
{2: (816, 83), 1: (729, 73)}
{2: (816, 83), 1: (727, 63), 3: (826, 113)}
{3: (831, 134), 1: (727, 63)}
{4: (722, 65)}
{4: (728, 66)}
{4: (719, 67)}
{4: (717, 72)}
{4: (715, 71)}
{4: (714, 71)}
{4: (712, 71)}
{4: (710, 70)}
{4: (708, 72)}
{4: (706, 71)}
{4: (703, 72)}
{4: (702, 73)}
{4: (700, 73)}
{4: (698, 74)}
{4: (696, 74)}
{4: (694, 75)}
{4: (692, 76)}
{4: (694, 77)}
{4: (689, 78)}
{4: (685, 78)}
{4: (684, 79)}
{4: (682, 79)}
{4: (685, 80)}
{4: (684, 81)}
{4: (675, 82)}
{4: (680, 83)}
{4: (671, 84)}
{4: (668, 84)}
{4: (666, 85)}
{4: (664, 86)}
{4: (662, 87)}
{4: (659, 87)}
{4: (657, 88)}
{4: (655, 89)}
{4: (652, 90)}
{4: (650, 91)}
{4: (655, 92)}
{4: (645, 93)}
{4: (643, 93)}
{4: (640, 94)}